# Portfolio Candidate Selection - Clean Implementation

This notebook implements the complete workflow for selecting optimal Martingale strategy parameters.

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Step 1: Load Data

In [5]:
csv_path = 'ZMarti_ReEntry_PT_BE_Optimizations.csv'
df = pd.read_csv(csv_path)
print(f"Loaded {len(df)} rows")
df.head()

Loaded 23808 rows


,Pass,Result,Profit,Expected Payoff,Profit Factor,Recovery Factor,Sharpe Ratio,Custom,PercentDD,Trades,ReEntryExtension,PTAsTimeREExt,CloseBreakEvenAfter,Pair
0,16027,95077314.0,-115442.60,-128.41224,0.05127,-0.97271,-1.59703,95077314,118.69,899,0.2,1.6,4,AUDCAD
1,16399,122077314.0,-115382.97,-130.96818,0.05112,-0.97223,-1.60310,122077314,118.68,881,0.2,1.7,4,AUDCAD
2,10075,73077314.0,-116450.94,-113.61067,0.03498,-0.98116,-1.66124,73077314,118.68,1025,0.2,1.6,3,AUDCAD
3,9703,72077114.0,-116355.39,-108.13698,0.02995,-0.98163,-1.66348,72077114,118.53,1076,0.2,1.5,3,AUDCAD
4,21979,129077114.0,-114689.55,-133.05052,0.06956,-0.96764,-1.56357,129077114,118.53,862,0.2,1.6,5,AUDCAD


## Step 2: Decode Result Column

The `Result` column contains three packed metrics:
- **Duration (Days)**: Scaled by 10 (AAAA)
- **Drawdown (%)**: Scaled by 10 (BBBB)
- **Re-entries**: (CC)

Structure: `(A * 1000000) + (B * 100) + C`

In [4]:
def decode_result(packed_double):
    """
    Decodes the Result column based on the AAAA BBBB CC structure.
    """
    packed = int(packed_double)
    
    # Extract components
    C = packed % 100
    B = (packed // 100) % 10000
    A = packed // 1000000
    
    # Scale back to original units
    duration_days = A / 10.0
    drawdown_percent = B / 10.0
    reentries = C
    
    return duration_days, drawdown_percent, reentries

# Apply decoding
df[["Duration", "Drawdown", "ReEntries"]] = df["Result"].apply(
    lambda x: pd.Series(decode_result(x))
)

# Drop original Result column and show preview
df = df.drop("Result", axis=1)
print("Decoded Result column")
df.head()

Decoded Result column


,Pass,Profit,Expected Payoff,Profit Factor,Recovery Factor,Sharpe Ratio,Custom,PercentDD,Trades,ReEntryExtension,PTAsTimeREExt,CloseBreakEvenAfter,Pair,Duration,Drawdown,ReEntries
0,16027,-115442.60,-128.41224,0.05127,-0.97271,-1.59703,95077314,118.69,899,0.2,1.6,4,AUDCAD,9.5,77.3,14.0
1,16399,-115382.97,-130.96818,0.05112,-0.97223,-1.60310,122077314,118.68,881,0.2,1.7,4,AUDCAD,12.2,77.3,14.0
2,10075,-116450.94,-113.61067,0.03498,-0.98116,-1.66124,73077314,118.68,1025,0.2,1.6,3,AUDCAD,7.3,77.3,14.0
3,9703,-116355.39,-108.13698,0.02995,-0.98163,-1.66348,72077114,118.53,1076,0.2,1.5,3,AUDCAD,7.2,77.1,14.0
4,21979,-114689.55,-133.05052,0.06956,-0.96764,-1.56357,129077114,118.53,862,0.2,1.6,5,AUDCAD,12.9,77.1,14.0
